In [ ]:
%matplotlib nbagg
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML, display
import matplotlib.image as img

plt.ioff()

In [ ]:
def ofc_update(avalanche,ofc_state,avhistory,t):
    retain = 0.25
    if not avalanche:
        kmax = np.amax(ofc_state)
        ofc_state += (1 -kmax)
        t += (1-kmax)
        avalanche = True
        avhistory = np.zeros(ofc_state.shape)
        return avalanche, ofc_state, avhistory,t
    if avalanche:
        mask_state = ofc_state * (ofc_state >= 1)
        avhistory = avhistory + (ofc_state >= 1)
        roll1 = np.pad(mask_state[1:,:],[(0,1),(0,0)],'constant')
        roll2 = np.pad(mask_state[:-1,:],[(1,0),(0,0)],'constant') 
        roll3 = np.pad(mask_state[:,1:],[(0,0),(0,1)],'constant') 
        roll4 = np.pad(mask_state[:,:-1],[(0,0),(1,0)],'constant')
        critsites = retain*(roll1+roll2+roll3+roll4)
        ofc_state[ofc_state>=1]=0
        ofc_state += critsites
        if np.amax(ofc_state) < 1:
            avalanche = False
        return avalanche, ofc_state, avhistory,t

In [ ]:
def updatefig(i):
    global avalanche
    global ofc_state
    global avhistory
    global avmags
    global aftershocks
    global t
    avalanche,ofc_state,avhistory,t = ofc_update(avalanche,ofc_state,avhistory,t)
    if not avalanche:
        avmag = int(np.sum(avhistory))
        ofc_avmags[:avmag] += 1
        aftershocks_t.append(t)
        aftershocks_m.append(avmag)
    im1.set_array(ofc_state)
    im2.set_array(avhistory)  
    plot1.set_data(aftershocks_t[-10:],aftershocks_m[-10:])
    ax3.set_ylim([0,np.amax(aftershocks_m[-10:])+1])
    ax3.set_xlim([np.amin(aftershocks_t[-10:])-0.001,np.amax(aftershocks_t[-10:])+0.001])
    plot2.set_ydata(ofc_avmags)
    ax4.set_ylim([0,np.amax(ofc_avmags)+1])
    return (im1,im2,plot1,ax1,plot2,ax2,)

In [ ]:
L = 20
ofc_state = np.random.rand(L,L)
avhistory = np.zeros(ofc_state.shape)
avalanche = False
ofc_avmags = np.zeros(500)
aftershocks_t=[0]
aftershocks_m=[0]
t=0

plt.close()
fig = plt.figure()
ax1 = plt.subplot2grid((2,2),(0,0))
ax1.set_title('Stan Układu')
ax1.tick_params(axis='both', which='both', bottom='off', top='off', right='off',left='off', labelbottom='off', labelleft = 'off')
im1 = plt.imshow(ofc_state, animated=True, cmap='plasma', interpolation='none')
ax2 = plt.subplot2grid((2,2),(0,1))
ax2.set_title('Obszar lawiny')
ax2.tick_params(axis='both', which='both', bottom='off', top='off', right='off',left='off', labelbottom='off', labelleft = 'off')
im2 = plt.imshow(ofc_state, animated=True, cmap='plasma', interpolation='none', vmax=4)
ax3 = plt.subplot2grid((2,2),(1,0))
plot1, = plt.plot([],[],'bx ')
ax3.set_xlim(0,2)
ax3.set_ylim(0,15)
ax3.set_title('Rozkład wielkości trzęsień')
ax3.set_ylabel('Skala trzęsienia')
ax3.set_xlabel('Czas')
ax4 = plt.subplot2grid((2,2),(1,1))
ax4.set_title('Rozkład wielkości trzęsień')
ax4.set_ylabel('Ilość wystąpień')
ax4.set_xlabel('Skala trzęsienia')
plot2, = plt.plot(np.random.rand(500),'bx ')

anim = animation.FuncAnimation(fig, updatefig, frames='none', interval=100)
plt.show()

In [ ]:
def bs_update(avalanche,bs_state,threshold,avlength):
    avlength+=1
    fmin = np.amin(bs_state)
    i, = np.where(bs_state == fmin)
    changed=[(i[0]+1)%bs_state.shape[0],i[0],i[0]-1]
    bs_state[changed] = np.random.rand()
    if np.amin(bs_state) < threshold:
        avalanche = True
        return avalanche,bs_state,threshold,changed,avlength
    else:
        threshold = np.amin(bs_state)
        avalanche = False
        return avalanche,bs_state,threshold,changed,avlength

In [ ]:
def updatefig(i):
    global avalanche
    global bs_state
    global threshold
    global avlength
    global avmags
    global pophistory
    avalanche,bs_state,threshold,changed,avlength = bs_update(avalanche,bs_state,threshold,avlength)
    pophistory[:-1,:] = pophistory[1:,:]
    pophistory[-1,:] = pophistory[-1,:] + 1
    pophistory[-1,changed] = 0
    if not avalanche:
        bs_avmags[:avlength] += 1
        avlength = 1
    line.set_ydata(bs_state)
    line2.set_ydata(threshold)
    im2.set_array(pophistory)
    plot1.set_ydata(bs_avmags)
    ax3.set_ylim([0,np.amax(bs_avmags)+1])
    return (line,line2,im2,ax3,plot1,)

In [ ]:
L = 200
bs_state = np.random.rand(L)
threshold = 0
avalanche = False
bs_avmags = np.zeros(500)
avlength = 0
pophistory = np.zeros((1000,L))

plt.close()
fig = plt.figure()
ax1 = plt.subplot2grid((2,2),(0,0))
ax1.set_title('Stan Układu')
ax1.set_ylabel('Wartość przystosowania')
ax1.set_xlabel('Gatunki')
line,=plt.plot(bs_state,'bx ')
line2 = plt.axhline(threshold,color='r')
ax2 = plt.subplot2grid((2,2),(0,1))
ax2.set_title('Historia Układu')
ax2.set_ylabel('Czas')
ax2.set_xlabel('Gatunki')
im2=plt.imshow(np.random.rand(1000,L), animated='True', cmap='plasma_r', interpolation='bilinear',aspect='auto',vmax=1000)
ax3 = plt.subplot2grid((2,2),(1,0),colspan=2)
ax3.set_title('Rozkład wielkości kaskad')
ax3.set_ylabel('Ilość wystąpień')
ax3.set_xlabel('Skala kaskady')
plot1, = plt.plot(np.random.rand(500),'bx ')

anim = animation.FuncAnimation(fig, updatefig, frames='none', interval=100)
plt.show()

In [ ]:
def runboth(iterations,bs_L,ofc_L):
    bs_state = np.random.rand(bs_L)
    ofc_state = np.random.rand(ofc_L,ofc_L)
    bs_avalanche = False
    ofc_avalanche = False
    threshold = 0
    avlength = 0
    avhistory = np.zeros(ofc_state.shape)
    bs_avmags = np.zeros(1000000)
    ofc_avmags = np.zeros(1000000)
    t = 0
    while True:
        if np.sum(bs_avmags) >= iterations and np.sum(ofc_avmags) >= iterations:
            break
        if np.sum(bs_avmags) < iterations:
            bs_avalanche,bs_state,threshold,changed,avlength = bs_update(bs_avalanche,bs_state,threshold,avlength)
            if not bs_avalanche:
                bs_avmags[:avlength] += 1
                avlength = 1
        if np.sum(ofc_avmags) < iterations:    
            ofc_avalanche,ofc_state,avhistory,t = ofc_update(ofc_avalanche,ofc_state,avhistory,t)
            if not ofc_avalanche:
                avmag = int(np.sum(avhistory))
                ofc_avmags[:avmag] += 1
    return ofc_avmags,bs_avmags
ofc_avmags, bs_avmags = runboth(10000,2500,30)

In [ ]:
def index_of_last_nonzero(lst):
    for i, value in enumerate(reversed(lst)):
        if value != 0:
            return len(lst)-i-1
    return -1
bs_avmags = bs_avmags[:index_of_last_nonzero(bs_avmags)]
ofc_avmags = ofc_avmags[:index_of_last_nonzero(ofc_avmags)]

In [ ]:
fig = plt.figure()
ax1 = plt.subplot2grid((1,2),(0,0))
plt.loglog(ofc_avmags,'bx ')
ax1.set_title('Olami-Feder-Christensen')
ax1.set_ylabel('Ilość wystąpień')
ax1.set_xlabel('Skala kaskady')
ax2 = plt.subplot2grid((1,2),(0,1))
plt.loglog(bs_avmags,'bx ')
ax2.set_title('Bak-Sneppen')
ax2.set_ylabel('Ilość wystąpień')
ax2.set_xlabel('Skala kaskady')
plt.show()

In [ ]:
fig.savefig('crits.png')